<a href="https://colab.research.google.com/github/codal-tshah/data-practices-2024/blob/29_apr/pyspark/pyspark_google_playstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7f9686f003477d2d0e767fb5dbb527cb84498c332ca3e83c7bbcc1e32932f32d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark




# *   Task to perform:
    find out TOP 10 reviews given to the apps
    Top paid apps
    Top paid rating apps
    Top 10 installs apps and distribution of type (free /paid)
    Category wise distribution of installed apps
    





In [5]:
from pyspark.sql import *
# from pyspark.sql.types import *
# from pyspark.sql.functions import *
spark = SparkSession.builder.appName("local.com").getOrCreate()

**Imported CSV FILE NORMALLY**

In [7]:
df = spark.read.csv("/content/drive/MyDrive/data_practices/pyspark/data/googleplaystore.csv")
df.printSchema()
df.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 _c0|           _c1|   _c2|    _c3| _c4|        _c5| _c6|  _c7|           _c8|                 _c9|              _c10|              _c11|        _c12|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Cate

**Creating DataFrame from CSV**

In [12]:
from html import escape
df = spark.read.load("/content/drive/MyDrive/data_practices/pyspark/data/googleplaystore.csv",format="csv",sep=",",escape='"' ,header=True,inferschema='true')

In [13]:
df.printSchema()
df.show()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+--------

count of rows

In [34]:
df.count()

10841

In [22]:
len(df.columns)

13

# Data Understanding and Cleaning

In [33]:
df.select([count(col(Reviews)).alias(Reviews)for Reviews in df.columns]).show()

+-----+--------+------+-------+-----+--------+-----+-----+--------------+------+------------+-----------+-----------+
|  App|Category|Rating|Reviews| Size|Installs| Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+-----+--------+------+-------+-----+--------+-----+-----+--------------+------+------------+-----------+-----------+
|10841|   10841| 10841|  10841|10841|   10841|10841|10841|         10840| 10841|       10841|      10840|      10840|
+-----+--------+------+-------+-----+--------+-----+-----+--------------+------+------------+-----------+-----------+



**Remove if any null value is there in data**

In [87]:
print("Number of duplicate records:", df.count() - df.dropDuplicates().count())

Number of duplicate records: 483


**Remove some unnecessary columns**

In [201]:
df1 = df.drop('Size','Last Updated', 'Current Ver', 'Android Ver' )
df1.printSchema()
df1.show()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)

+--------------------+--------------+------+-------+-----------+----+-----+--------------+--------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|Content Rating|              Genres|
+--------------------+--------------+------+-------+-----------+----+-----+--------------+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|      Everyone|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|      Ever

In [202]:
df1.count()

10841

In [203]:
unique_apps_count = df1.dropDuplicates(['App']).count()

# Show the count of unique apps
print("Number of unique apps:", unique_apps_count)

Number of unique apps: 9660


In [204]:
df1 = df1.dropDuplicates()
df1.count()

10356

In [205]:
df1.show()

+--------------------+-------------------+------+--------+--------------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|      Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+--------------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1,000,000,000+|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|   50,000,000+|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923|      100,000+|Free|    0|    Mature 17+|              Dating|
|  Domino's Pizza USA|     FOOD_AND_DRINK|   4.7| 1032935|   10,000,000+|Free|    0|      Everyone|        Food & Drink|
|      Subway Surfers|               GAME|   4.5|27722264|1,000,000,000+|Free|    0|  Everyone 10+|              Arcade|
|FINAL FANTASY BRA...|          

In [206]:
df1.select("Category").distinct().count()

34

In [207]:
df1.select("Category").distinct().show()

+-------------------+
|           Category|
+-------------------+
|             EVENTS|
|             COMICS|
|             SPORTS|
|            WEATHER|
|      VIDEO_PLAYERS|
|  AUTO_AND_VEHICLES|
|          PARENTING|
|      ENTERTAINMENT|
|    PERSONALIZATION|
| HEALTH_AND_FITNESS|
|   TRAVEL_AND_LOCAL|
|BOOKS_AND_REFERENCE|
|     FOOD_AND_DRINK|
|        PHOTOGRAPHY|
|           BUSINESS|
|             FAMILY|
|           SHOPPING|
|     HOUSE_AND_HOME|
|               GAME|
|                1.9|
+-------------------+
only showing top 20 rows



In [208]:
df.select('Type').distinct().show()

+----+
|Type|
+----+
|   0|
| NaN|
|Free|
|Paid|
+----+



In [209]:
null_type = df1.filter(col('Type')=='NaN')
null_type.show()
zero_value = df1.filter(col('Type')==0)
zero_value.show()
remove_type = ['null_type', 'zero_value']



+--------------------+--------+------+-------+--------+----+-----+--------------+--------+
|                 App|Category|Rating|Reviews|Installs|Type|Price|Content Rating|  Genres|
+--------------------+--------+------+-------+--------+----+-----+--------------+--------+
|Command & Conquer...|  FAMILY|   NaN|      0|       0| NaN|    0|  Everyone 10+|Strategy|
+--------------------+--------+------+-------+--------+----+-----+--------------+--------+

+--------------------+--------+------+-------+--------+----+--------+--------------+-----------------+
|                 App|Category|Rating|Reviews|Installs|Type|   Price|Content Rating|           Genres|
+--------------------+--------+------+-------+--------+----+--------+--------------+-----------------+
|Life Made WI-Fi T...|     1.9|  19.0|   3.0M|    Free|   0|Everyone|          NULL|February 11, 2018|
+--------------------+--------+------+-------+--------+----+--------+--------------+-----------------+



Highest number of categories apps used

In [210]:
df.groupBy("Category").count().orderBy(col("count").desc()).show(5)

+--------+-----+
|Category|count|
+--------+-----+
|  FAMILY| 1972|
|    GAME| 1144|
|   TOOLS|  843|
| MEDICAL|  463|
|BUSINESS|  460|
+--------+-----+
only showing top 5 rows



In [211]:
df1.show()

+--------------------+-------------------+------+--------+--------------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|      Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+--------------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1,000,000,000+|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|   50,000,000+|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923|      100,000+|Free|    0|    Mature 17+|              Dating|
|  Domino's Pizza USA|     FOOD_AND_DRINK|   4.7| 1032935|   10,000,000+|Free|    0|      Everyone|        Food & Drink|
|      Subway Surfers|               GAME|   4.5|27722264|1,000,000,000+|Free|    0|  Everyone 10+|              Arcade|
|FINAL FANTASY BRA...|          

In [212]:
df1.select("Rating").summary().show()

+-------+------+
|summary|Rating|
+-------+------+
|  count| 10356|
|   mean|   NaN|
| stddev|   NaN|
|    min|   1.0|
|    25%|   4.1|
|    50%|   4.4|
|    75%|   4.6|
|    max|   NaN|
+-------+------+



In [213]:
df1.select("Reviews").summary().show()

+-------+------------------+
|summary|           Reviews|
+-------+------------------+
|  count|             10356|
|   mean| 405963.4421052632|
| stddev|2697034.9198568026|
|    min|                 0|
|    25%|              32.0|
|    50%|            1667.0|
|    75%|           46353.0|
|    max|              9992|
+-------+------------------+



In [214]:
df1.groupby("Category").agg(avg("Rating").alias("AveragedRating")).show()

+-------------------+-----------------+
|           Category|   AveragedRating|
+-------------------+-----------------+
|             EVENTS|              NaN|
|             COMICS|              NaN|
|             SPORTS|              NaN|
|            WEATHER|              NaN|
|      VIDEO_PLAYERS|              NaN|
|  AUTO_AND_VEHICLES|              NaN|
|          PARENTING|              NaN|
|      ENTERTAINMENT|4.136036036036036|
|    PERSONALIZATION|              NaN|
|   TRAVEL_AND_LOCAL|              NaN|
| HEALTH_AND_FITNESS|              NaN|
|BOOKS_AND_REFERENCE|              NaN|
|     FOOD_AND_DRINK|              NaN|
|        PHOTOGRAPHY|              NaN|
|           BUSINESS|              NaN|
|             FAMILY|              NaN|
|           SHOPPING|              NaN|
|     HOUSE_AND_HOME|              NaN|
|               GAME|              NaN|
|                1.9|             19.0|
+-------------------+-----------------+
only showing top 20 rows



In [215]:
df1.groupby("Category").agg(count("Rating").alias("AveragedRating")).show()

+-------------------+--------------+
|           Category|AveragedRating|
+-------------------+--------------+
|             EVENTS|            64|
|             COMICS|            60|
|             SPORTS|           351|
|            WEATHER|            82|
|      VIDEO_PLAYERS|           175|
|  AUTO_AND_VEHICLES|            85|
|          PARENTING|            60|
|      ENTERTAINMENT|           111|
|    PERSONALIZATION|           388|
|   TRAVEL_AND_LOCAL|           237|
| HEALTH_AND_FITNESS|           306|
|BOOKS_AND_REFERENCE|           230|
|     FOOD_AND_DRINK|           124|
|        PHOTOGRAPHY|           322|
|           BUSINESS|           427|
|             FAMILY|          1943|
|           SHOPPING|           223|
|     HOUSE_AND_HOME|            80|
|               GAME|          1121|
|                1.9|             1|
+-------------------+--------------+
only showing top 20 rows



In [216]:
result = df1.groupby("Category").agg(count("Rating").alias("AveragedRating"))

# Order the result in descending order by the aggregated column "AveragedRating"
result = result.orderBy("AveragedRating", ascending=False)

# Show the result
result.show()


+-------------------+--------------+
|           Category|AveragedRating|
+-------------------+--------------+
|             FAMILY|          1943|
|               GAME|          1121|
|              TOOLS|           842|
|           BUSINESS|           427|
|            MEDICAL|           408|
|       PRODUCTIVITY|           407|
|    PERSONALIZATION|           388|
|          LIFESTYLE|           373|
|      COMMUNICATION|           366|
|            FINANCE|           360|
|             SPORTS|           351|
|        PHOTOGRAPHY|           322|
| HEALTH_AND_FITNESS|           306|
|             SOCIAL|           280|
| NEWS_AND_MAGAZINES|           264|
|   TRAVEL_AND_LOCAL|           237|
|BOOKS_AND_REFERENCE|           230|
|           SHOPPING|           223|
|             DATING|           196|
|      VIDEO_PLAYERS|           175|
+-------------------+--------------+
only showing top 20 rows



In [217]:
df1.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)



In [218]:
df1 = df1.withColumn("Installs", regexp_replace('Installs', '\+', ''))
df1.show()

+--------------------+-------------------+------+--------+-------------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|     Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+-------------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1,000,000,000|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|   50,000,000|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923|      100,000|Free|    0|    Mature 17+|              Dating|
|  Domino's Pizza USA|     FOOD_AND_DRINK|   4.7| 1032935|   10,000,000|Free|    0|      Everyone|        Food & Drink|
|      Subway Surfers|               GAME|   4.5|27722264|1,000,000,000|Free|    0|  Everyone 10+|              Arcade|
|FINAL FANTASY BRA...|               GAM

In [220]:
df2 = df1.withColumn("Installs", regexp_replace('Installs', '\,', ''))
df2.show()

+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|  Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1000000000|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|  50000000|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923|    100000|Free|    0|    Mature 17+|              Dating|
|  Domino's Pizza USA|     FOOD_AND_DRINK|   4.7| 1032935|  10000000|Free|    0|      Everyone|        Food & Drink|
|      Subway Surfers|               GAME|   4.5|27722264|1000000000|Free|    0|  Everyone 10+|              Arcade|
|FINAL FANTASY BRA...|               GAME|   4.6|  745684|   500

In [249]:
df2 = df2.withColumn('Installs', col('Installs').cast(IntegerType()))
df2.printSchema()
df2.show()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)

+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|  Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1000000000|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|  50000000|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923| 

In [250]:
df3 = df2.withColumn('Rating', col('Rating').cast(FloatType()))
df3.printSchema()
df3.show()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)

+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|  Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1000000000|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|  50000000|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923|  

In [251]:
df3 = df3.withColumn('Reviews', col('Reviews').cast(IntegerType()))
df3.printSchema()
df3.show()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)

+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|                 App|           Category|Rating| Reviews|  Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------------+------+--------+----------+----+-----+--------------+--------------------+
|            Hangouts|      COMMUNICATION|   4.0| 3419249|1000000000|Free|    0|      Everyone|       Communication|
|            Contacts|      COMMUNICATION|   4.3|   66602|  50000000|Free|    0|      Everyone|       Communication|
|BiggerCity: Chat ...|             DATING|   4.1|     923| 

In [252]:
df3.orderBy('Reviews', ascending=False).show()

+--------------------+-------------+------+--------+----------+----+-----+--------------+--------------------+
|                 App|     Category|Rating| Reviews|  Installs|Type|Price|Content Rating|              Genres|
+--------------------+-------------+------+--------+----------+----+-----+--------------+--------------------+
|            Facebook|       SOCIAL|   4.1|78158306|1000000000|Free|    0|          Teen|              Social|
|            Facebook|       SOCIAL|   4.1|78128208|1000000000|Free|    0|          Teen|              Social|
|  WhatsApp Messenger|COMMUNICATION|   4.4|69119316|1000000000|Free|    0|      Everyone|       Communication|
|  WhatsApp Messenger|COMMUNICATION|   4.4|69109672|1000000000|Free|    0|      Everyone|       Communication|
|           Instagram|       SOCIAL|   4.5|66577446|1000000000|Free|    0|          Teen|              Social|
|           Instagram|       SOCIAL|   4.5|66577313|1000000000|Free|    0|          Teen|              Social|
|

# ** top 10 reviews given to the apps**

In [253]:
q= df3.orderBy(desc('Reviews')).select("App", "Reviews").limit(10)
q.show()

+--------------------+--------+
|                 App| Reviews|
+--------------------+--------+
|            Facebook|78158306|
|            Facebook|78128208|
|  WhatsApp Messenger|69119316|
|  WhatsApp Messenger|69109672|
|           Instagram|66577446|
|           Instagram|66577313|
|           Instagram|66509917|
|Messenger – Text ...|56646578|
|Messenger – Text ...|56642847|
|      Clash of Clans|44893888|
+--------------------+--------+



In [254]:
df3.count()

10356

Remove duplicates

In [266]:
df4 = df3.dropDuplicates(subset=['App','Category','Rating','Installs' ])
df4.show()

+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|Content Rating|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|"i DT" Fútbol. To...|             SPORTS|   NaN|     27|     500|Free|    0|      Everyone|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1000000|Free|    0|      Everyone|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|   10000|Free|    0|    Mature 17+|           Comics|
|                  .R|              TOOLS|   4.5|    259|   10000|Free|    0|      Everyone|            Tools|
|              /u/app|      COMMUNICATION|   4.7|    573|   10000|Free|    0|    Mature 17+|    Communication|
|              058.ba| NEWS_AND_MAGAZINES|   4.4|     27|     100|Free|    0|      Everyone| News & Magazines|
|

In [256]:
q= df4.orderBy(desc('Reviews')).select("App", "Reviews").limit(10)
q.show()

+--------------------+--------+
|                 App| Reviews|
+--------------------+--------+
|            Facebook|78158306|
|  WhatsApp Messenger|69119316|
|           Instagram|66577313|
|Messenger – Text ...|56642847|
|      Clash of Clans|44891723|
|      Clash of Clans|44881447|
|Clean Master- Spa...|42916526|
|      Subway Surfers|27722264|
|             YouTube|25655305|
|Security Master -...|24900999|
+--------------------+--------+



# **top paid apps**

In [258]:
df4.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)



In [268]:
df5 = df4.withColumn('Price', regexp_replace('Price', '\$', ''))
df5.show()

+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|Content Rating|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|"i DT" Fútbol. To...|             SPORTS|   NaN|     27|     500|Free|    0|      Everyone|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1000000|Free|    0|      Everyone|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|   10000|Free|    0|    Mature 17+|           Comics|
|                  .R|              TOOLS|   4.5|    259|   10000|Free|    0|      Everyone|            Tools|
|              /u/app|      COMMUNICATION|   4.7|    573|   10000|Free|    0|    Mature 17+|    Communication|
|              058.ba| NEWS_AND_MAGAZINES|   4.4|     27|     100|Free|    0|      Everyone| News & Magazines|
|

In [270]:
df5 = df5.withColumn('Price', col('Price').cast(FloatType()))
df5.printSchema()
df5.show()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)

+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|Content Rating|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|"i DT" Fútbol. To...|             SPORTS|   NaN|     27|     500|Free|  0.0|      Everyone|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1000000|Free|  0.0|      Everyone|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|   10000|Free|  0.0|    Mature 17

In [275]:
top_paid = df5.filter(df5['Type']=='Paid').orderBy(desc('Price')).select('App', 'Installs', 'Price').limit(20)
top_paid.show()

+--------------------+--------+------+
|                 App|Installs| Price|
+--------------------+--------+------+
|I'm Rich - Trump ...|   10000| 400.0|
|           I am Rich|    5000|399.99|
|  I AM RICH PRO PLUS|    1000|399.99|
|   I Am Rich Premium|   50000|399.99|
|       I Am Rich Pro|    5000|399.99|
|I'm Rich/Eu sou R...|       0|399.99|
|      I am Rich Plus|   10000|399.99|
|          I am Rich!|    1000|399.99|
|           I am rich|  100000|399.99|
|I am rich (Most e...|    1000|399.99|
|  I am rich(premium)|    5000|399.99|
|most expensive ap...|     100|399.99|
|         💎 I'm rich|   10000|399.99|
|         Eu Sou Rico|       0|394.99|
|           I Am Rich|   10000|389.99|
| I am extremely Rich|    1000|379.99|
|       I am rich VIP|   10000|299.99|
|        EP Cook Book|       0| 200.0|
|       cronometra-br|       0|154.99|
|BP Fitness Lead S...|       1|109.99|
+--------------------+--------+------+



# Top Installed apps which is paid

In [281]:
top_installed_paid = df4.filter(df1["Type"] == "Paid").orderBy(desc("Installs")).select("App", "Installs").limit(20)
top_installed_paid.show()

+--------------------+--------+
|                 App|Installs|
+--------------------+--------+
|       Hitman Sniper|10000000|
|           Minecraft|10000000|
| DraStic DS Emulator| 1000000|
|Card Wars - Adven...| 1000000|
|   Cut the Rope GOLD| 1000000|
|             Threema| 1000000|
| Facetune - For Free| 1000000|
|Five Nights at Fr...| 1000000|
| Fruit Ninja Classic| 1000000|
|Grand Theft Auto:...| 1000000|
|          HD Widgets| 1000000|
|         Bloons TD 5| 1000000|
|Infinity Dungeon VIP| 1000000|
|League of Stickma...| 1000000|
|Lost Journey (Dre...| 1000000|
|Beautiful Widgets...| 1000000|
|Sleep as Android ...| 1000000|
|Stickman Legends:...| 1000000|
|              Tasker| 1000000|
|          True Skate| 1000000|
+--------------------+--------+



In [282]:
df5.show()

+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|Content Rating|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|"i DT" Fútbol. To...|             SPORTS|   NaN|     27|     500|Free|  0.0|      Everyone|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1000000|Free|  0.0|      Everyone|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|   10000|Free|  0.0|    Mature 17+|           Comics|
|                  .R|              TOOLS|   4.5|    259|   10000|Free|  0.0|      Everyone|            Tools|
|              /u/app|      COMMUNICATION|   4.7|    573|   10000|Free|  0.0|    Mature 17+|    Communication|
|              058.ba| NEWS_AND_MAGAZINES|   4.4|     27|     100|Free|  0.0|      Everyone| News & Magazines|
|

# **top paid rating apps**

In [287]:
df5.select("Rating").distinct().show()

+------+
|Rating|
+------+
|   5.0|
|   2.5|
|   3.8|
|   2.2|
|   4.6|
|   3.4|
|   4.2|
|   1.8|
|   3.2|
|   2.9|
|   2.0|
|   3.6|
|   3.0|
|   1.5|
|   2.6|
|   3.3|
|   2.8|
|   4.4|
|   3.5|
|   1.0|
+------+
only showing top 20 rows



In [294]:
df6 = df5.filter((col("Type") == "Paid") & (col("Rating") != "NaN"))
df6.show()

+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|Content Rating|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+--------------+-----------------+
|10 WPM Amateur ha...|      COMMUNICATION|   3.5|     10|     100|Paid| 1.49|      Everyone|    Communication|
|10,000 Quotes DB ...|BOOKS_AND_REFERENCE|   4.1|     70|     500|Paid| 0.99|      Everyone|Books & Reference|
|17th Edition Cabl...|BOOKS_AND_REFERENCE|   4.4|     47|    1000|Paid| 3.08|      Everyone|Books & Reference|
|2 Amateur ham rad...|      COMMUNICATION|   4.8|      6|     100|Paid| 1.49|      Everyone|    Communication|
|2017 EMRA Antibio...|            MEDICAL|   4.4|     12|    1000|Paid|16.99|      Everyone|          Medical|
|20WPM Amateur ham...|      COMMUNICATION|   4.3|     12|     100|Paid| 1.49|      Everyone|    Communication|
|

In [295]:
df6.count()

606

In [297]:
top_paid_rating_apps = df6.filter(df1["Type"] == "Paid").orderBy(desc("Rating")).select("App", "Rating").limit(20)
top_paid_rating_apps.show()

+--------------------+------+
|                 App|Rating|
+--------------------+------+
|AJ Gray Dark Icon...|   5.0|
|              211:CK|   5.0|
|30WPM Amateur ham...|   5.0|
| AC DC Power Monitor|   5.0|
|        ADS-B Driver|   5.0|
|   AJ Blue Icon Pack|   5.0|
|AP Art History Fl...|   5.0|
|Android P Style I...|   5.0|
|        BC MVA Fines|   5.0|
|Hey AJ! It's Satu...|   5.0|
|         CA Speakers|   5.0|
|CI 174 Gray Icon ...|   5.0|
|       Chronolink DX|   5.0|
|    DL Image Manager|   5.0|
|Easy Hotspot Ad Free|   5.0|
|      FHR 5-Tier 2.0|   5.0|
|            FO Bixby|   5.0|
|Hey AJ! It's Bedt...|   5.0|
|        Morse Player|   5.0|
|             Mu.F.O.|   5.0|
+--------------------+------+



# **top 10 installs apps **

In [298]:
top_10_installs = df5.orderBy(desc("Installs")).select("App", "Installs", "Type").limit(10)
top_10_installs.show()



+--------------------+----------+----+
|                 App|  Installs|Type|
+--------------------+----------+----+
|              Google|1000000000|Free|
|  Google Street View|1000000000|Free|
|Google Chrome: Fa...|1000000000|Free|
|            Facebook|1000000000|Free|
|        Google Drive|1000000000|Free|
|   Google Play Books|1000000000|Free|
|   Google Play Games|1000000000|Free|
|               Gmail|1000000000|Free|
|   Google Play Games|1000000000|Free|
|       Google Photos|1000000000|Free|
+--------------------+----------+----+

+----+-----+
|Type|count|
+----+-----+
|   0|    1|
| NaN|    1|
|Free| 8997|
|Paid|  758|
+----+-----+



In [299]:
type_distribution = df5.groupBy("Type").count()
type_distribution.show()


+----+-----+
|Type|count|
+----+-----+
|   0|    1|
| NaN|    1|
|Free| 8997|
|Paid|  758|
+----+-----+



In [300]:
category_distribution = df1.groupBy("Category").count().orderBy(desc("count"))
category_distribution.show()

+-------------------+-----+
|           Category|count|
+-------------------+-----+
|             FAMILY| 1943|
|               GAME| 1121|
|              TOOLS|  842|
|           BUSINESS|  427|
|            MEDICAL|  408|
|       PRODUCTIVITY|  407|
|    PERSONALIZATION|  388|
|          LIFESTYLE|  373|
|      COMMUNICATION|  366|
|            FINANCE|  360|
|             SPORTS|  351|
|        PHOTOGRAPHY|  322|
| HEALTH_AND_FITNESS|  306|
|             SOCIAL|  280|
| NEWS_AND_MAGAZINES|  264|
|   TRAVEL_AND_LOCAL|  237|
|BOOKS_AND_REFERENCE|  230|
|           SHOPPING|  223|
|             DATING|  196|
|      VIDEO_PLAYERS|  175|
+-------------------+-----+
only showing top 20 rows



In [304]:
print("Correlation between installs and ratings:", df6.stat.corr("Installs", "Rating"))


Correlation between installs and ratings: 0.06048990562269225


In [306]:
print("Correlation between installs and Price:", df5.stat.corr("Installs", "Price"))


Correlation between installs and Price: -0.00961491394199247
